In [1]:
import numpy as np
import cv2 
import keras
import pandas as pd
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as c
import sys
import shutil
import os
import keras 
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from keras.models import load_model

In [3]:
import pathlib
import imageio

training_paths = pathlib.Path('/home/kanik/Documents/dataset3/images/').glob('*.jpg')
training_sorted = sorted([str(x) for x in training_paths])

In [4]:
def make_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

In [5]:
green_count = 0
red_count = 0
other_count = 0
training_size = 100
test_size = 70
training_images = []
test_images = []
training_labels =[]
test_labels = []



green_dir_train  ='/home/kanik/Documents/dataset3/color_green/train/green/'
red_dir_train = '/home/kanik/Documents/dataset3/color_red/train/red/'
green_dir_val = '/home/kanik/Documents/dataset3/color_green/validation/green'
red_dir_val = '/home/kanik/Documents/dataset3/color_red/validation/red'
other_dir_train  ='/home/kanik/Documents/dataset3/color_other/train/other/'
other_dir_val = '/home/kanik/Documents/dataset3/color_other/validation/other'


make_dir(green_dir_train)
make_dir(red_dir_train)
make_dir(green_dir_val)
make_dir(red_dir_val)
make_dir(other_dir_train)
make_dir(other_dir_val)

def getZeros(number):
    if(number>10 and number<100):
        return "0"
    if (number<10):
        return "00"
    else:
        return ""
    
for i , file in enumerate(training_sorted):
    if training_sorted[i][38] == 'g':
        if green_count >=training_size+test_size:
            continue
        else :
            image = c.imread(file)
            #image = c.resize(image,(size,size),interpolation = c.INTER_AREA)
            if green_count < training_size:
                training_images.append(image)
                training_labels.append(0)
                zeros = getZeros(green_count)
                c.imwrite(green_dir_train + 'green' +str(zeros)+str(green_count)+'.jpg',image)
            if green_count >=training_size and green_count <training_size+test_size:
                test_images.append(image)
                test_labels.append(0)
                zeros = getZeros(green_count - 1000)
                c.imwrite(green_dir_val + 'green' +str(zeros)+str(green_count)+'.jpg',image)
        green_count +=1
    if training_sorted[i][38] == 'r':
        if red_count >=training_size+test_size:
            continue
        else :
            image = c.imread(file)
            #image = c.resize(image,(size,size),interpolation = c.INTER_AREA)
            if red_count < training_size:
                training_images.append(image)
                training_labels.append(1)
                zeros = getZeros(red_count)
                c.imwrite(red_dir_train + 'red' +str(zeros)+str(red_count)+'.jpg',image)
            if red_count >=training_size and red_count <training_size+test_size:
                test_images.append(image)
                test_labels.append(1)
                zeros = getZeros(red_count - 1000)
                c.imwrite(red_dir_val + "red" +str(zeros)+str(red_count)+'.jpg',image)
        red_count +=1
    
    if training_sorted[i][38] == 'o':
        if other_count >=training_size+test_size:
            continue
        else :
            image = c.imread(file)
            #image = c.resize(image,(size,size),interpolation = c.INTER_AREA)
            if other_count < training_size:
                training_images.append(image)
                training_labels.append(2)
                zeros = getZeros(other_count)
                c.imwrite(other_dir_train + 'other' +str(zeros)+str(other_count)+'.jpg',image)
            if other_count >=training_size and other_count <training_size+test_size:
                test_images.append(image)
                test_labels.append(2)
                zeros = getZeros(other_count - 1000)
                c.imwrite(other_dir_val + "other" +str(zeros)+str(other_count)+'.jpg',image)
        other_count +=1
                

    if green_count == training_size+test_size+1 and red_count == training_size+test_size+1 and other_count == training_size+test_size+1:
        break
        
print('Training and Test data Extraction complete')        

Training and Test data Extraction complete


In [6]:
np.savez('red_vs_green_vs_other_training_data.npz',np.array(training_images))
np.savez('red_vs_green_vs_other_training_labels.npz',np.array(training_labels))
np.savez('red_vs_green_vs_other_test_data.npz',np.array(test_images))
np.savez('red_vs_green_vs_other_test_labels.npz',np.array(test_labels))

In [7]:
def load_data_training_and_test(datasetname):
    npzfile = np.load(datasetname + '_training_data.npz')
    train = npzfile['arr_0']
    
    npzfile = np.load(datasetname + '_training_labels.npz')
    train_labels = npzfile['arr_0']
    
    npzfile = np.load(datasetname + '_test_data.npz')
    test = npzfile['arr_0']
    
    npzfile = np.load(datasetname + '_test_labels.npz')
    test_labels = npzfile['arr_0']
    
    
    return(train,train_labels),(test,test_labels)

In [9]:
for i in range(1,11):
    random = np.random.randint(0,len(training_images))
    c.imshow('image_'+str(i),training_images[random])
    if training_labels[random] ==1:
        print(str(i)+'-red')
    elif training_labels[random] ==0 :
        print(str(i)+'-green')
    else :
        print(str(i)+'-other')
    c.waitKey(0)
    
c.destroyAllWindows() 

1-green
2-green
3-red
4-other
5-other
6-green
7-other
8-red
9-other
10-red


In [10]:
from keras.utils import np_utils
batch_size=16
epochs=15
(x_train,y_train),(x_test,y_test)=load_data_training_and_test('red_vs_green_vs_other')
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
#y_train=y_train.reshape(y_train.shape[0],1)
#y_test=y_test.reshape(y_test.shape[0],1)
y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)

x_train/=255
x_test/=255
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(300, 1, 1, 3)
(300, 3)
(210, 1, 1, 3)
(210, 3)


In [11]:
from keras.optimizers import SGD
model=Sequential() #use to make layers
model.add(Conv2D(32,kernel_size=(1,1),activation='relu',input_shape=x_train.shape[1:]))
model.add(Conv2D(64,(1,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=SGD(0.01),metrics=['accuracy'])
print(model.summary())

W0726 21:24:05.126200 139879870719808 deprecation_wrapper.py:119] From /home/kanik/anaconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 21:24:05.145822 139879870719808 deprecation_wrapper.py:119] From /home/kanik/anaconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 21:24:05.149250 139879870719808 deprecation_wrapper.py:119] From /home/kanik/anaconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 21:24:05.207322 139879870719808 deprecation_wrapper.py:119] From /home/kanik/anaconda3/envs/PythonCPU/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 1, 32)          128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          2112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [12]:
history=model.fit(x_train,y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test,y_test))
#batch size here is how many nodes in hidden layer
score=model.evaluate(x_test,y_test,verbose=0)
print('Test losses',score[0])
print('Test accuracy',score[1])

W0726 21:24:06.223574 139879870719808 deprecation.py:323] From /home/kanik/anaconda3/envs/PythonCPU/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 300 samples, validate on 210 samples
Epoch 1/15
300/300 [==============================] - 1s 3ms/step - loss: 1.0980 - acc: 0.3467 - val_loss: 1.0583 - val_acc: 0.6667
Epoch 2/15
300/300 [==============================] - 0s 971us/step - loss: 1.0601 - acc: 0.5000 - val_loss: 1.0371 - val_acc: 1.0000
Epoch 3/15
300/300 [==============================] - 0s 842us/step - loss: 1.0328 - acc: 0.6367 - val_loss: 1.0166 - val_acc: 0.9333
Epoch 4/15
300/300 [==============================] - 0s 823us/step - loss: 1.0053 - acc: 0.7033 - val_loss: 0.9983 - val_acc: 0.9333
Epoch 5/15
300/300 [==============================] - 0s 881us/step - loss: 0.9869 - acc: 0.7833 - val_loss: 0.9798 - val_acc: 0.8762
Epoch 6/15
300/300 [==============================] - 0s 1ms/step - loss: 0.9597 - acc: 0.7933 - val_loss: 0.9603 - val_acc: 0.8333
Epoch 7/15
300/300 [==============================] - 0s 1ms/step - loss: 0.9429 - acc: 0.8300 - val_loss: 0.9406 - val_acc: 0.8000
Epoch 8/15
300/300 [==

In [13]:
model.save('colors.h5')

In [14]:
classifier = load_model('colors.h5')

def draw_test(name, pred, input_im):
    print(pred)
    BLACK = [0,0,0]
    print(type(pred))
    if pred == "0":
        pred = "green"
    if pred == "1":
        pred = "red"
    if pred == "2":
        pred = "others"
    print(pred)
    '''   
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0] ,
                                        cv2.BORDER_CONSTANT,value=BLACK)
    #expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (252, 70) , 
                cv2.FONT_HERSHEY_COMPLEX_SMALL,4, (0,255,0), 2)
    cv2.imshow(name, expanded_image)
    '''

for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]

    imageL = cv2.resize(input_im, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
    cv2.imshow("Test Image", imageL)

    input_im = input_im.reshape(1,1,1,3) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])

    draw_test("Prediction", res,input_im) 
    cv2.waitKey(0)

cv2.destroyAllWindows()


1
<class 'str'>
red
1
<class 'str'>
red
1
<class 'str'>
red
1
<class 'str'>
red
1
<class 'str'>
red
1
<class 'str'>
red
2
<class 'str'>
others
1
<class 'str'>
red
0
<class 'str'>
green
0
<class 'str'>
green


In [15]:
classifier = load_model('colors.h5')
def new(fr):
    global classifier
    res = str(classifier.predict_classes(fr, 1, verbose = 0)[0])
    #print(res)
    return res

In [17]:
import cv2 

img=cv2.imread('/home/kanik/Downloads/image.jpg')
row=img.shape[0]
col=img.shape[1]
red=0
green=0
others=0
frame1 = img.astype('float32')
frame1 /= 255
for i in range(row):
    for j in range(col):
        a=new(frame1[i][j].reshape(1,1,1,3))
        if(a=='0'):
            green=green+1
        elif(a=='1'):
            red=red+1
        elif(a=='2'):
            others=others+1
print(red)
print(green)
print(others)

20243
9502
20689


In [18]:
import cv2 
import numpy as np
from keras.models import load_model 

classifier=load_model('colors.h5')
def draw_test(name, pred, input_im):
    BLACK = [0,0,0]
    if pred == "[0]":
        pred = "green"
    if pred == "[1]":
        pred = "red"
    if pred == "[2]":
        pred=="others"
    cv2.putText(input_im, str(pred), (180, 70) , 
                cv2.FONT_HERSHEY_COMPLEX_SMALL,2, (0,255,0), 2)
    cv2.imshow(name,input_im)
   


In [19]:
cam = cv2.VideoCapture(0)
while cam.isOpened():
    ret, frame= cam.read(0)
    frames=cv2.resize(frame,(1,1),interpolation=cv2.INTER_AREA)
    img_rows=frames[0].shape[0]
    img_cols=frames[0].shape[1]
    frames=frames.reshape(1,frames.shape[0],img_rows,img_cols)
    frames=frames.astype('float32')
    frames/=255

    print(frames.shape)
    
    pred=str(classifier.predict_classes(frames,verbose=0)[0])
    print(pred)
    draw_test("prediction",pred,frame)
    if cv2.waitKey(1) == 27 :
        break
    
cam.release()
cv2.destroyAllWindows()

(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 3)
0
(1, 1, 1, 

In [19]:
classifier = load_model('colors.h5')
def old(fr):
    global classifier
    res = str(classifier.predict_classes(fr, 1, verbose = 0)[0])
    #print(res)
    return res

In [20]:
import cv2
import numpy as np
from keras.models import load_model

classifier=load_model('colors.h5')

def f_fun(frames):
    red=0
    green=0
    others=0
    
    #frames=frames.reshape(1,frames.shape[0],img_rows,img_cols)
    #frames=cv2.resize(frame,(1,1),interpolation=cv2.INTER_AREA)
    frames=frames.astype('float32')
    frames/=255
    img_rows=frames.shape[0]
    img_cols=frames.shape[1]
    print(img_rows)
    print(img_cols)
    for i in range(img_rows):
        for j in range(img_cols):
            a=old(frames[i][j].reshape(1,1,1,3))
            if(a=='0'):
                frames[i][j] = [0,1,0]
            #elif(a=='1'):
                #green=green+1
            #elif(a=='2'):
                #others=others+1
        print('1st')
    print('2nd')
    #cv2.imshow('name',frames)
    frames *= 255
    frames = frames.astype('uint8')
    return frames
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read(0)
    cv2.imshow('Original',frame)
    if cv2.waitKey(1)==ord('s'):
        cv2.destroyAllWindows()
        print('strat')
        x = f_fun(frame)
        print('completed')
        cv2.imshow('x',x)
        break
    if cv2.waitKey(1) == 27:
        break
    
cap.release()



strat
480
640
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1st
1s

In [22]:
cap.release()